In [ ]:
import torch
device = (
    "cuda" if torch.cuda.is_available()
    else "cpu"
)
print(f"using {device}")

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

data_mnist = datasets.MNIST(
    root= "data", # Folder
    train= True, # True: 60.000 images
    download= True,
    transform=ToTensor() # to tensors
)

In [ ]:
figure = plt.figure(figsize=(8,8))
rows, columns = 3, 3

for i in range(1, columns * rows + 1):
    sample_idx = torch.randint(len(data_mnist), size=(1,)).item()

    img, label = data_mnist[sample_idx]

    figure.add_subplot(rows, columns, i)
    plt.title(str(label))
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")

plt.show()

In [ ]:
train, val, test = torch.utils.data.random_split(
    data_mnist, [0.8, 0.1, 0.1]
)

print(f"Train size: {len(train)}")
print(f"Val size: {len(val)}")
print(f"test size: {len(test)}")

In [ ]:
from torch import nn

class NeuronalNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        self.flatten = nn.Flatten() # image flatten
        self.net = nn.Sequential(
            nn.Linear(28*28, 15), # init step, hidden step
            nn.ReLU(), # ReLU function activation
            nn.Linear(15,10), # output step, no function activation 
        )

    def forward(self, x):
        x = self.flatten(x) # data flatten
        logits = self.net(x) # prediction

        return logits


In [ ]:
model = NeuronalNetwork().to(device)

In [ ]:
total_params = sum(p.numel() for p in model.parameters())

In [ ]:
img, lbl = train[200]

In [ ]:
lbl = torch.tensor(lbl).reshape(1)


In [ ]:
img, lbl = img.to(device), lbl.to(device)

In [ ]:
logits = model(img)
print(logits)

In [ ]:
y_pred = logits.argmax(1)

plt.imshow(img.cpu().squeeze(), cmap="gray")

print(f"Logits: {logits}")
print(f"Prediction: {y_pred[0]}")
print(f"Category: {lbl[0]}")

In [ ]:
fn_lost = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)

In [ ]:
loss = fn_lost(logits, lbl)
print(loss) # near to 0

In [ ]:
loss.backward()

In [ ]:
optimizer.step() # update parameters
optimizer.zero_grad() # clear grad

# TRAINING

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 1000

# training data
train_loader = DataLoader(
    dataset=train,
    batch_size=BATCH_SIZE,
    shuffle=True # random mix
)

# validation data
val_loader = DataLoader(
    dataset=val,
    batch_size=BATCH_SIZE,
    shuffle=False
)


In [ ]:
LEARNING_RATE = 0.1
EPOCH = 10 # iteration

In [ ]:
fn_lost = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):

    train_size = len(dataloader.dataset)
    nbatch = len(dataloader)

    model.train()

    lost_train, accuracy = 0, 0 # lost down, accuracy up to 100%

    for nlote, (X, y) in enumerate(dataloader):
        
        X, y = X.to(device), y.to(device)

        logits = model(X)

        loss = loss_fn(logits, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        lost_train += loss.item()
        accuracy += (logits.argmax(1)==y).type(torch.float).sum().item()

        if nlote % 10 == 0:

            ndatos = nbatch*BATCH_SIZE

            print(f"Loss: {loss.item():>7f} [{ndatos:>5d}/{train_size:>5d}]")

    lost_train /= nbatch
    accuracy /= train_size

    print(f"Average accuracy:")
    print(f"Training: {(100*accuracy):>0.1f}% / {lost_train:>8f}")

In [ ]:
def val_loop(dataloader, model, loss_fn):

    val_size = len(dataloader.dataset)
    nbatch = len(dataloader)

    model.eval()

    lost_val, accuracy = 0, 0

    with torch.no_grad():
        for X, y in dataloader:

            X, y = X.to(device), y.to(device)

            logits = model(X)

            lost_val += loss_fn(logits, y).item()
            accuracy += (logits.argmax(1) == y).type(torch.float).sum().item()

    lost_val /= nbatch
    accuracy /= val_size

    print(f"Validation: {(100*accuracy):>0.1f}% / {lost_val:>8f} \n")

In [ ]:
for t in range(EPOCH):
    print(f"Iteration: {t+1}/{EPOCH}\n--------------")

    train_loop(train_loader, model, fn_lost, optimizer)
    val_loop(val_loader, model, fn_lost)

print("The model was training!")

In [ ]:
def predict(model, img):
    
    logits = model(img)
    y_pred = logits.argmax(1).item()

    plt.imshow(img.cpu().squeeze(), cmap="gray")
    plt.title(f"Category: {y_pred}")

In [ ]:
img, lbl = test[1234]

predict(model, img)